# Analysis of `vehicle-summary.xml`

In [ ]:
import pprint, math, datetime
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

# normal = "charlottenburg-meso"
# scenarios = [
#   "charlottenburg-no-meso",
#   "charlottenburg-no-meso-periodic"
# ]

# normal = "berlin-1pct-with-internal-links"
# scenarios = [
#   "berlin-1pct-without-internal-links"
# ]

normal = "../aws/N/output"
scenarios = [
  # "apats-output"
  # "../simulation/charlottenburg/mesosim"
]



In [ ]:
def get_vehicle_step_summary(file_path):
    steps = []
    
    for event, element in etree.iterparse(file_path, tag="step"):
        current_step = {
            "time": float(element.attrib["time"]),
            "running": float(element.attrib["running"]),
            "meanWaitingTime": float(element.attrib["meanWaitingTime"]),
            "meanTravelTime": float(element.attrib["meanTravelTime"]),
            "meanSpeed": float(element.attrib["meanSpeed"]),
            "meanSpeedRelative": float(element.attrib["meanSpeedRelative"])
        }
        steps.append(current_step)

        element.clear()

    return steps

def aggregate_by(values, interval=60):
    interval_steps = []
    interval_values = []

    interval_sum = 0
    for t in range(len(values)):
        interval_sum += values[t]

        if t > 0 and t % interval == 0:
            interval_steps.append(t - interval)
            interval_values.append(interval_sum / interval)
            interval_sum = 0

        # if t == len(values) - 1 and interval_sum > 0:
        #     interval_steps.append(math.ceil(t / interval))
        #     interval_values.append(interval_sum / interval)
        #     interval_sum = 0

    return (interval_steps, interval_values)

n_steps = get_vehicle_step_summary(f"{normal}/vehicle-summary.xml")

scenario_data = {}

for s in scenarios:
    steps = get_vehicle_step_summary(f"{s}/vehicle-summary.xml")
    scenario_data[s] = steps

### Compare mean travelTime/speed over time

In [ ]:
def plot_attribute(attr, interval=60*60):
    fig, ax = plt.subplots()

    mean_values = list(map(lambda s: s[attr], n_steps))
    x, y = aggregate_by(mean_values, interval=interval)
    ax.plot(x, y, label=normal)

    for s in scenarios:
        steps = scenario_data[s]
        mean_values = list(map(lambda s: s[attr], steps))
        x, y = aggregate_by(mean_values, interval=interval)
        ax.plot(x, y, label=s)

    ax.set_xlabel("Time of Day")
    xticks = x[0::3]
    xticks_labels = list(map(lambda t: strftime("%H:%M:%S", gmtime(t)), xticks))
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticks_labels)

    ax.set_ylabel(attr)
    ax.legend(loc="upper right")

    fig.set_dpi(100)
    fig.savefig(f"{attr}.png", bbox_inches="tight")

interval = 60*60
plot_attribute("meanSpeed", interval=interval)
plot_attribute("meanTravelTime", interval=interval)
plot_attribute("running", interval=interval)